# Data Analysis with PySpark

## What is PySpark?

As you know, Python is a dynamic, general-purpose language. PySpark provides an entry point to Python in the computational model of Spark. PySpark is open source and it is fast, expressive, and versatile. 

Spark is coded in Scala. However, you can use it with R and Java programming languages. 

All three major cloud (Amazon Web Services, Google Cloud Platform, and Microsoft Azure) allows you to work with PySpark. 

## Hands-on Data Analysis with PySpark

This section shows you how to perform a simple NLP analysis with PySpark. Before reading the dataset, you need to create a `SparkSession`. Let's import the SparkSession entry point is located in the `pyspark.sql`. This entry point allows the functionality for data transformation.

In [1]:
from pyspark.sql import SparkSession 

Note that `SparkSession` is an object, so it has methods associated with it. Let's instantiate SparkSession in order to use PySpark:

In [2]:
spark = (SparkSession
         .builder
         .appName("Analyzing the Vocabulary of the Sonnets.")
         .getOrCreate())

Here, a `builder` offers a set of methods to create a highly configurable object and the `getOrCreate` method provides both interactive and batch mode and it help you to avoid the creation of a new SparkSession if one already exists. 

Spark entry point is `sparkContext`. `SparkSession` is a superset of that. In essence, SparkContext is the linl between your Python REPL and the Spark cluster. Let's take a look at the `sparkContext`.

In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=Analyzing the Vocabulary of The Sonnets.>

## RDD vs DataFrame

Before loading the dataset, I want to discuss the RDD (resilient distributed dataset) and data frame structures. PySpark allows you to use  two main structures for storing data when performing manipulations: The RDD and the DataFrame. You can think of the RNN as the most basic abstraction in Spark. It is an immutable distributed collection of objects (or rows). Each record in the RDD is an independent entity. 

In a nutshell, you can think of a Spark DataFrame as a Pandas DataFrame or a relational database table with rows and named columns. However, a Spark DataFrame uses the memory of several machines instead of a single machine. In this tutorial, I'm going to work with the DataFrames.

My goal in this data analysis is to find most popular words used in Shakespeare' the sonnets book with PySpark. As you know, data analysis consists of three major steps: reading, transforming, and exporting. Let's get started with data ingestion.

## Data Ingestion

The dataset I'm going to use is Shakespeare' the sonnets book from Kaggle. Kaggle is a fantastic source for interesting (and free) datasets for research and education. You can access this dataset here. Let's read the dataset with the `spark.read.test` method.

In [3]:
df = spark.read.text("shakespeare.txt")

Let's take a look at the structure of this variable.

In [4]:
df

DataFrame[value: string]

As expected, This is a `DataFrame`. Here, you can also see the name of the columns and their type. Please note that PySpark `DataFrames` consist of a collection of columns. You can use the `printSchema` method to quick overview of features datatype. Let's now use the `printSchema` to display the schema in a tree form.

In [5]:
df.printSchema()

root
 |-- value: string (nullable = true)



You can see the same information with the data frame’s `dtypes` attribute. Let me show you this.

In [19]:
df.dtypes

[('value', 'string')]

You can also use the `toPandas()` method to return Pyspark DataFrame as Pandas table.

In [6]:
df.limit(10).toPandas()

,value
0,THE SONNETS
1,
2,by William Shakespeare
3,
4,"From fairest creatures we desire increase,"
5,"That thereby beauty's rose might never die,"
6,"But as the riper should by time decease,"
7,His tender heir might bear his memory:
8,"But thou contracted to thine own bright eyes,"
9,Feed'st thy light's flame with self-substantia...


## Exploring Data

It's important to explore data before analyzing data. If you can understand your dataset, you can easily find the patterns in the dataset. The `show` method displays a few rows of the data. Let's take a look at the rows of our data with the `show` method. It shows 20 rows and truncate long values by default.

In [7]:
df.show()

+--------------------+
|               value|
+--------------------+
|         THE SONNETS|
|                    |
|by William Shakes...|
|                    |
|From fairest crea...|
|That thereby beau...|
|But as the riper ...|
|His tender heir m...|
|But thou contract...|
|Feed'st thy light...|
|Making a famine w...|
|Thy self thy foe,...|
|Thou that art now...|
|And only herald t...|
|Within thine own ...|
|And tender churl ...|
|Pity the world, o...|
|To eat the world'...|
|                    |
|When forty winter...|
+--------------------+
only showing top 20 rows



You can set the number of row and truncate. Let me show you this.

In [8]:
df.show(10, truncate=60)

+-----------------------------------------------------+
|                                                value|
+-----------------------------------------------------+
|                                          THE SONNETS|
|                                                     |
|                               by William Shakespeare|
|                                                     |
|           From fairest creatures we desire increase,|
|          That thereby beauty's rose might never die,|
|             But as the riper should by time decease,|
|               His tender heir might bear his memory:|
|        But thou contracted to thine own bright eyes,|
|Feed'st thy light's flame with self-substantial fuel,|
+-----------------------------------------------------+
only showing top 10 rows



## Column transformations

As mentioned, my goal is to count the most used words in the dataset. To do this, I'm going to perform covers as following steps:

1. Token — Tokenize each word.
2. Clean — Remove any punctuation and/or tokens that aren’t words and then lowercase each word.
3. Count — Count the frequency of each word present in the text.

To tokenize each word, I'm going to set the `select` method. You can use the `select` method to select one or more columns from your DataFrame like a `SELECT` statement in SQL. Let's get the `value` column with the `col` method and then transform the sentences into the words as an array column with the `split` method. You can also rename transformation column with the `alias` method. Please keep in mind that PySpark’s data preprocessing methods that operate on columns is located in `pyspark.sql.functions`. 

In [11]:
import pyspark.sql.functions as F
lines = df.select(F.split(F.col("value"), " ").alias("line"))
lines.show(5)

+--------------------+
|                line|
+--------------------+
|      [THE, SONNETS]|
|                  []|
|[by, William, Sha...|
|                  []|
|[From, fairest, c...|
+--------------------+
only showing top 5 rows



### Reshaping your data

PySpark can have columns of nested values, like arrays of elements. The `explode` method allows you to extract the elements into distinct records. Let's create one record for each word with the `explode` function.

In [13]:
words = lines.select(F.explode(F.col("line")).alias("word"))
words.show(10)

+-----------+
|       word|
+-----------+
|        THE|
|    SONNETS|
|           |
|         by|
|    William|
|Shakespeare|
|           |
|       From|
|    fairest|
|  creatures|
+-----------+
only showing top 10 rows



## Working with words

In this section, I'm going to show you how to perform lower the case via the `lower` method and remove punctuation via a regular expression. Let's lower the case of all the words in the dataframe with the `lower` method first.

In [14]:
words_lower = words.select(F.lower(F.col("word")).alias("word_lower"))
words_lower.show(10)

+-----------+
| word_lower|
+-----------+
|        the|
|    sonnets|
|           |
|         by|
|    william|
|shakespeare|
|           |
|       from|
|    fairest|
|  creatures|
+-----------+
only showing top 10 rows



After that let's clean our words of any punctuation and other non-useful characters the `regexp_extract` method.

In [15]:
words_clean = words_lower.select(F.regexp_extract(F.col("word_lower"), "[a-z]+", 0).alias("word"))
words_clean.show(10)

+-----------+
|       word|
+-----------+
|        the|
|    sonnets|
|           |
|         by|
|    william|
|shakespeare|
|           |
|       from|
|    fairest|
|  creatures|
+-----------+
only showing top 10 rows



## Filtering rows

You can filter DataFrames using either SQL-style `where` syntax or with `Column` objects. To filter records according to a certain condition, you can use the `where` or `filter` methods. These methods allows you to provide a test that will return `True` or `False` and only the records returning `True` will be kept. Now I'm going to filter the no null words using comparison operators using the `filter` method.

In [16]:
words_nonull = words_clean.filter(F.col("word") != "")
words_nonull.show(10)

+-----------+
|       word|
+-----------+
|        the|
|    sonnets|
|         by|
|    william|
|shakespeare|
|       from|
|    fairest|
|  creatures|
|         we|
|     desire|
+-----------+
only showing top 10 rows



Nice, the blank cell is gone. Let's now find only the words with more than five characters.

In [18]:
words_great_three_char = words_clean.filter(length(col("word")) > 5)
words_great_three_char.show(10)

+-----------+
|       word|
+-----------+
|    sonnets|
|    william|
|shakespeare|
|    fairest|
|  creatures|
|     desire|
|   increase|
|    thereby|
|     beauty|
|     should|
+-----------+
only showing top 10 rows



##  Grouping records

To count the number of each word, we can group the records. The easiest way to group the record is to use the `groupby` method. Let's only create a groupby object with the `groupby` method.

In [17]:
groups = words_nonull.groupby(F.col("word"))
print(groups)

The `groupby` method returns a `GroupedData` object that waits for an aggregation method. Using this `GroupedData` object, we can calculate the frequency of the words with the `count` method.

In [20]:
results = words_nonull.groupby(F.col("word")).count()
print(results)

DataFrame[word: string, count: bigint]


Awesome, we counted the frequency of the words. Let's take a look at the number of these words.

In [21]:
results.show(10)

+-------+-----+
|   word|count|
+-------+-----+
|    art|   52|
|   some|   31|
|  those|   33|
|  still|   41|
|   hope|    6|
| travel|    3|
|  cures|    1|
| ransom|    2|
|  spoil|    1|
|tresses|    1|
+-------+-----+
only showing top 10 rows



## Ordering the results with orderBy

The `orderBy()` method allows you to order a DataFrame by the values of one or many columns. There are two ways to order. You can use the column names as parameters or can set the `col` function. Let's take a look at these ways.

In [23]:
results.orderBy("count", ascending=False).show(10)

+----+-----+
|word|count|
+----+-----+
| and|  490|
| the|  432|
|  to|  414|
|  my|  393|
|  of|  370|
|   i|  351|
|  in|  323|
|that|  323|
| thy|  287|
|thou|  235|
+----+-----+
only showing top 10 rows



To order values, you can also use the `col` method via the `decs` method.

In [24]:
results.orderBy(col("count").desc()).show(10)

+----+-----+
|word|count|
+----+-----+
| and|  490|
| the|  432|
|  to|  414|
|  my|  393|
|  of|  370|
|   i|  351|
|  in|  323|
|that|  323|
| thy|  287|
|thou|  235|
+----+-----+
only showing top 10 rows



## Writing data

So far we saw the data on the screen, but you can also want to export your results. To do this, you can use the `write` method. Let's export our results in comma-separated value (CSV) file that is a human-readable format.

In [22]:
results.write.csv("my_count.csv")

AnalysisException: path file:/D:/Videolar/Tirendaz/Books-Notebook-Notes/Pyspark/Hands-on PySpark/my_count.csv already exists.;

Note that PySpark creates one file per partition. This is due to PySpark is worked multiple workers. So you have many partitions. Let's take a look at these partitions.

In [26]:
ls my_count.csv

 Volume in drive D is Yeni Birim
 Volume Serial Number is B6A7-CDC9

 Directory of D:\Videolar\Tirendaz\Books-Notebook-Notes\Pyspark\Hands-on PySpark\my_count.csv

08/18/2022  08:50 AM    <DIR>          .
08/18/2022  08:50 AM    <DIR>          ..
08/18/2022  08:50 AM                 8 ._SUCCESS.crc
08/18/2022  08:50 AM                12 .part-00000-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv.crc
08/18/2022  08:50 AM                12 .part-00001-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv.crc
08/18/2022  08:50 AM                12 .part-00002-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv.crc
08/18/2022  08:50 AM                12 .part-00003-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv.crc
08/18/2022  08:50 AM                12 .part-00004-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv.crc
08/18/2022  08:50 AM                12 .part-00005-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv.crc
08/18/2022  08:50 AM                12 .part-00006-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.c

08/18/2022  08:50 AM                95 part-00051-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               139 part-00052-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               170 part-00053-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               120 part-00054-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM                90 part-00055-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               130 part-00056-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               159 part-00057-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               119 part-00058-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               170 part-00059-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               117 part-00060-50f14a9a-3887-4e87-ae73-bbbf8de9d3cf-c000.csv
08/18/2022  08:50 AM               157 p

You can reduce the number of partitions with the `coalesce` method and set your desired number of partitions. I'm going to put one file per partition.

In [27]:
results.coalesce(1).write.csv("my_single_partition.csv")

Let's take a look at this file.

In [28]:
ls my_single_partition.csv

 Volume in drive D is Yeni Birim
 Volume Serial Number is B6A7-CDC9

 Directory of D:\Videolar\Tirendaz\Books-Notebook-Notes\Pyspark\Hands-on PySpark\my_single_partition.csv

08/18/2022  08:51 AM    <DIR>          .
08/18/2022  08:51 AM    <DIR>          ..
08/18/2022  08:51 AM                 8 ._SUCCESS.crc
08/18/2022  08:51 AM               252 .part-00000-9feb981b-09b0-41d5-a9e2-1c0f04dc8a17-c000.csv.crc
08/18/2022  08:51 AM                 0 _SUCCESS
08/18/2022  08:51 AM            30,959 part-00000-9feb981b-09b0-41d5-a9e2-1c0f04dc8a17-c000.csv
               4 File(s)         31,219 bytes
               2 Dir(s)  84,648,456,192 bytes free


Nice we wrote our record in a CSV file.

## Conclusion

In this notebook, I showed you how to use PySpark for analyzing a data. I hope you enjoy it. 

Don't forget to follow us on [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎